In [2]:
import warnings
warnings.filterwarnings("ignore")

!pip install pandas -q

import pandas as pd
# Хак чтобы работало на Anaconda под MacOs 10.15.7 Catalina
pd.DataFrame.iteritems = pd.DataFrame.items

In [3]:
!pip install plotly -q

import plotly.graph_objects as go

def linear_plot(df, title):
    fig = go.Figure([go.Scatter(x=df['date'], y=df['close'], mode='lines')])
    fig.update_layout(plot_bgcolor='white', 
                      xaxis_title='Date',
                      yaxis_title='Price',
                      title=title)
    fig.show()

def candlestick_plot(df, title):
    fig = go.Figure([go.Candlestick(x=df['date'], 
                                open=df['open'],
                                high=df['high'],
                                low=df['low'],
                                close=df['close'])])
    fig.update_layout(xaxis_rangeslider_visible=False,
                      plot_bgcolor='white',
                      xaxis_title='Date',
                      yaxis_title='Price',
                      title=title)
    fig.update_yaxes(fixedrange=False)
    fig.show()

In [71]:
!pip install ccxt -q

import ccxt
import time

exchange = ccxt.binance()
symbol = 'ETH/USDT'
timeframe = '1h'
since = exchange.parse8601('2024-01-01T00:00:00Z')
all_ohlcvs = []

while True:
    try:
        ohlcvs = exchange.fetch_ohlcv(symbol, timeframe, since)
        all_ohlcvs += ohlcvs
        if len(ohlcvs):
            print('Fetched', len(ohlcvs), symbol, timeframe, 'candles from', exchange.iso8601(ohlcvs[0][0]))
            since = ohlcvs[-1][0] + 1
            sleep_interval = exchange.rateLimit / 1000
            print('Sleep for', sleep_interval)
            time.sleep(sleep_interval)
        else:
            break
    except Exception as e:
        print(type(e).__name__, str(e))
print('Fetched', len(all_ohlcvs), symbol, timeframe, 'candles in total')

Fetched 500 ETH/USDT 1h candles from 2024-01-01T00:00:00.000Z
Sleep for 0.05
Fetched 500 ETH/USDT 1h candles from 2024-01-21T20:00:00.000Z
Sleep for 0.05
Fetched 500 ETH/USDT 1h candles from 2024-02-11T16:00:00.000Z
Sleep for 0.05
Fetched 500 ETH/USDT 1h candles from 2024-03-03T12:00:00.000Z
Sleep for 0.05
Fetched 500 ETH/USDT 1h candles from 2024-03-24T08:00:00.000Z
Sleep for 0.05
Fetched 500 ETH/USDT 1h candles from 2024-04-14T04:00:00.000Z
Sleep for 0.05
Fetched 500 ETH/USDT 1h candles from 2024-05-05T00:00:00.000Z
Sleep for 0.05
Fetched 500 ETH/USDT 1h candles from 2024-05-25T20:00:00.000Z
Sleep for 0.05
Fetched 500 ETH/USDT 1h candles from 2024-06-15T16:00:00.000Z
Sleep for 0.05
Fetched 500 ETH/USDT 1h candles from 2024-07-06T12:00:00.000Z
Sleep for 0.05
Fetched 500 ETH/USDT 1h candles from 2024-07-27T08:00:00.000Z
Sleep for 0.05
Fetched 500 ETH/USDT 1h candles from 2024-08-17T04:00:00.000Z
Sleep for 0.05
Fetched 500 ETH/USDT 1h candles from 2024-09-07T00:00:00.000Z
Sleep for 0.05

In [73]:
df = pd.DataFrame(all_ohlcvs)
df.columns = ['date','open','high','low','close','volume']
df = df.sort_values(by='date')
df = df.drop_duplicates(subset='date').reset_index(drop=True)
df['date'] = pd.to_datetime(df['date'], unit='ms')
df.to_csv('ETH_USDT_1h.csv', index=False)
df

,date,open,high,low,close,volume
0,2024-01-01 00:00:00,2281.87,2297.18,2281.27,2295.51,10771.9183
1,2024-01-01 01:00:00,2295.52,2306.60,2292.90,2303.72,8413.4260
2,2024-01-01 02:00:00,2303.72,2304.72,2291.20,2293.02,5808.2533
3,2024-01-01 03:00:00,2293.03,2294.50,2271.00,2273.81,10745.7360
4,2024-01-01 04:00:00,2273.80,2279.86,2265.24,2279.55,9681.5762
...,...,...,...,...,...,...
8504,2024-12-20 08:00:00,3390.48,3390.79,3280.00,3290.29,98189.8031
8505,2024-12-20 09:00:00,3290.29,3315.39,3260.35,3265.59,84933.0510
8506,2024-12-20 10:00:00,3265.60,3278.00,3145.65,3186.73,142491.2678
8507,2024-12-20 11:00:00,3186.73,3218.21,3103.00,3104.56,132908.4020
